<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%jsroot on
TCanvas c;
TFile f("corrsi25.root");
TF1 *sfit; //TF1指针定义在外部，避免定义在smart()内部时出现变量冲突。

In [2]:
TString fit(Double_t a,Double_t b,TString name){
    TString rtn;
    rtn.Form("Double_t %s(Double_t *x,Double_t *par){if(x[0]>%f&&x[0]<%f){TF1::RejectPoint();return 0;}return par[0]+par[1]*x[0];}",name.Data(),a,b);
    return rtn;
}//返回函数的字符串，避免每次手动修改函数脚本。

In [3]:
void smartfit(TH1F *h,Double_t fitl,Double_t fitr,Double_t cutl,Double_t cutr,TString name,TString option=""){
    //h为参与拟合的一维谱，fitl，fitr为参与拟合的总区间范围，cutl，cutr为不参与拟合的区间，name为拟合函数的名称。多次调用本函数时，应避免name重复。
    gROOT->ProcessLine(fit(cutl,cutr,name));
    //函数指针的方式实现动态调整函数参数比较麻烦，暂无法实现。采用gROOT则可以很简单地实现该目的。
    TString cmd;
    cmd.Form("sfit=new TF1(\"sfit\",%s,0,5000,2)",name.Data());
    gROOT->ProcessLine(cmd.Data());
    //终端环境下，cmd字符串内的name内容会被识别为函数名，但脚本中name表示的是字符串。或许可以用其他方式来实现该目的，但目前这种已足够简便。
    h->Fit("sfit",option,"",fitl,fitr);
}

In [4]:
TString fitexp(Double_t a,Double_t b,TString name){
    TString rtn;
    rtn.Form("Double_t %s(Double_t *x,Double_t *par){if(x[0]>%f&&x[0]<%f){TF1::RejectPoint();return 0;}return TMath::Exp(par[0]+par[1]*x[0])+par[2];}",name.Data(),a,b);
    return rtn;
}//返回函数的字符串，避免每次手动修改函数脚本。

In [5]:
void smartfitexp(TH1F *h,Double_t fitl,Double_t fitr,Double_t cutl,Double_t cutr,TString name,TString option){
    //h为参与拟合的一维谱，fitl，fitr为参与拟合的总区间范围，cutl，cutr为不参与拟合的区间，name为拟合函数的名称。多次调用本函数时，应避免name重复。
    gROOT->ProcessLine(fitexp(cutl,cutr,name));
    //函数指针的方式实现动态调整函数参数比较麻烦，暂无法实现。采用gROOT则可以很简单地实现该目的。
    TString cmd;
    cmd.Form("sfit=new TF1(\"sfit\",%s,0,5000,3)",name.Data());
    gROOT->ProcessLine(cmd.Data());
    //终端环境下，cmd字符串内的name内容会被识别为函数名，但脚本中name表示的是字符串。或许可以用其他方式来实现该目的，但目前这种已足够简便。
    h->Fit("sfit","","",fitl,fitr);
}

In [6]:
Double_t fit1(Double_t *x,Double_t *p){
  if(x[0]>220*5&&x[0]<5000-220*5){
    TF1::RejectPoint();
    return 0;
    }
    return p[0]*exp(-x[0]*log(2)/p[1])+p[2];
}

In [7]:
void decayfit1(TH1F *h){
  TF1 *f1=new TF1("f1",fit1,0,5000,3);
  f1->SetParameters(exp(6.5004),log(2)/8e-3,23);
  h->Fit(f1);
  Double_t *p=f1->GetParameters();
  Double_t p0=p[0];
  Double_t p1=p[1];
  Double_t p2=p[2];
  cout<<"有效峰面积： "<<h->Integral(0,220)-220*p[2]<<endl;
}

In [8]:
tree->Draw("bea>>h1(1000,0,10000)","cond==1&&bz==2&&ve[0]<200&&dtime<220*5");
tree->Draw("bea>>h2(1000,0,10000)","cond==1&&bz==2&&ve[0]<200&&dtime>5000-220*5");
h1->Sumw2();
h1->Add(h2,-1);
h1->Draw();
c.Draw();

In [9]:
tree->Draw("gea>>g1(8000,0,4000)","cond==1&&bz==2&&ve[0]<200&&dtime<220*5&&bea>910&&bea<980");
tree->Draw("gea>>g2(8000,0,4000)","cond==1&&bz==2&&ve[0]<200&&dtime>5000-220*5&&bea>910&&bea<980");
g1->Sumw2();
g1->Add(g2,-1);
c.Clear();
smartfit(g1,1250,1450,1355,1375,"m1","L");
g1->Draw();
c.Draw();

(TF1 *) 0x7f9898c52050
 FCN=188.433 FROM MIGRAD    STATUS=CONVERGED     107 CALLS         108 TOTAL
                     EDM=2.98334e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.47694e+00   4.10046e-01   2.93489e-04  -4.05773e-03
   2  p1          -1.56304e-03   3.00149e-04   2.14841e-07  -5.13185e+00
                               ERR DEF= 0.5


In [10]:
cout<<g1->Integral(2710,2750)<<'\n'<<(1+2750-2710)*( 2.47694-1.56304e-03*(2750+2710.0)/4)<<endl;

442
14.079


In [11]:
c.Clear();
h1->Draw();
smartfitexp(h1,600,1200,900,1060,"l2","L");
c.Draw();

(TF1 *) 0x7f9898cf5460
 FCN=54.3244 FROM MIGRAD    STATUS=CONVERGED     226 CALLS         227 TOTAL
                     EDM=7.37435e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.38939e+00   1.36204e-01   3.12591e-05   2.97112e-03
   2  p1          -3.07794e-03   2.50106e-04   3.97360e-08   2.35530e+00
   3  p2           4.64008e+02   5.85579e+01   2.61112e-02   3.76346e-06


In [12]:
TF1 *k=new TF1("k","expo+pol0",0,8000);
k->SetParameters(9.38939,-3.07794e-03,4.64008e+02);

In [13]:
cout<<h1->Integral(92,98)<<'\n'<<k->Integral(920,980)/10<<endl;

18346
3916.74


In [14]:
tree->Draw("gea>>g3(8000,0,4000)","cond==1&&bz==2&&ve[0]<200&&dtime<220*5&&bea>2110&&bea<2200");
tree->Draw("gea>>g4(8000,0,4000)","cond==1&&bz==2&&ve[0]<200&&dtime>5000-220*5&&bea>2110&&bea<2200");
g3->Sumw2();
g3->Add(g4,-1);
c.Clear();
smartfit(g3,1250,1450,1355,1375,"m2","L");
g3->Draw();
c.Draw();

(TF1 *) 0x7f9898ea8dd0
 FCN=135.736 FROM MIGRAD    STATUS=CONVERGED     111 CALLS         112 TOTAL
                     EDM=1.60238e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.91643e+00   2.97308e-01   1.93339e-04   1.66517e-02
   2  p1          -1.24621e-03   2.16489e-04   1.40793e-07   2.29385e+01
                               ERR DEF= 0.5


In [15]:
cout<<g3->Integral(2710,2750)<<'\n'<<(1+2750-2710)*( 1.91643-1.24621e-03*(2750+2710.0)/4)<<endl;

354
8.82949


In [16]:
c.Clear();
h1->Draw();
smartfit(h1,2060,2240,2090,2230,"l5","L");
c.Draw();

(TF1 *) 0x7f9898e91220
 FCN=2.02782 FROM MIGRAD    STATUS=CONVERGED     115 CALLS         116 TOTAL
                     EDM=1.10465e-07    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   1.5 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           4.37740e+02   3.03988e+02  -1.92881e+00   1.27120e-05
   2  p1          -1.68976e-02   1.43632e-01   9.83309e-04   2.98926e-02
                               ERR DEF= 0.5


In [17]:
cout<<h1->Integral(212,220)<<'\n'<<(1+220-212)*(4.37740e+02-1.68976e-02*(220+212.0)*5)<<endl;

11078
3611.17


In [18]:
h1->GetXaxis()->SetRangeUser(1700,2900);
h1->ShowBackground(50);
cout<<h1_background->Integral(212,220)<<endl;
c.Draw();

2462.24


In [19]:
h1->GetXaxis()->UnZoom();

In [20]:
tree->Draw("gea>>g5(8000,0,4000)","cond==1&&bz==2&&ve[0]<200&&dtime<220*5&&bea>3410&&bea<3520");
tree->Draw("gea>>g6(8000,0,4000)","cond==1&&bz==2&&ve[0]<200&&dtime>5000-220*5&&bea>3410&&bea<3520");
g5->Sumw2();
g5->Add(g6,-1);
c.Clear();
smartfit(g5,1250,1450,1355,1375,"m3","L");
g5->Draw();
c.Draw();

(TF1 *) 0x7f989906fc20
 FCN=159.71 FROM MIGRAD    STATUS=CONVERGED     111 CALLS         112 TOTAL
                     EDM=9.2752e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           3.41030e+00   4.05696e-01   2.63388e-04   4.76313e-03
   2  p1          -2.24909e-03   2.95807e-04   1.92044e-07   7.42192e+00
                               ERR DEF= 0.5


In [21]:
cout<<g5->Integral(2710,2750)<<'\n'<<(1+2750-2710)*(3.41030-2.24909e-03*(2750+2710.0)/4)<<endl;

489
13.952


In [22]:
cout<<h1->Integral(342,352)<<'\n'<<566*11<<endl;

16063
6226


In [23]:
tree->Draw("gea>>g7(8000,0,4000)","cond==1&&bz==2&&ve[0]<200&&dtime<220*5&&bea>4200&&bea<4300");
tree->Draw("gea>>g8(8000,0,4000)","cond==1&&bz==2&&ve[0]<200&&dtime>5000-220*5&&bea>4200&&bea<4300");
g7->Sumw2();
g7->Add(g8,-1);
c.Clear();
smartfit(g7,1250,1450,1355,1375,"m4","L");
g7->Draw();
c.Draw();

(TF1 *) 0x7f989912bbc0
 FCN=219.985 FROM MIGRAD    STATUS=CONVERGED     117 CALLS         118 TOTAL
                     EDM=2.40493e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.16095e+01   6.58628e-01   4.89135e-04   4.78459e-04
   2  p1          -7.80473e-03   4.75907e-04   3.53447e-07   5.69205e-01
                               ERR DEF= 0.5


In [24]:
cout<<g7->Integral(2710,2750)<<'\n'<<(1+2750-2710)*( 1.16095e+01-7.80473e-03*(2750+2710.0)/4)<<endl;

1603
39.1978


In [25]:
c.Clear();
h1->Draw();
smartfit(h1,4000,5300,4200,5100,"l3","L");
c.Draw();

(TF1 *) 0x7f9898c3c3b0
 FCN=47.9832 FROM MIGRAD    STATUS=CONVERGED     138 CALLS         139 TOTAL
                     EDM=1.64257e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.89872e+02   1.32626e+01   7.16686e-03   3.36136e-05
   2  p1          -4.20101e-02   2.75490e-03   1.48871e-06   1.17978e-01
                               ERR DEF= 0.5


In [26]:
cout<<h1->Integral(421,430)<<'\n'<<(1+430-421)*(2.89872e+02-4.20101e-02*(430+421.0)*5)<<endl;

47073
1111.19


In [27]:
tree->Draw("le1>>h3(1000,0,10000)","cond==1&&bz==1&&dtime<220*5");
tree->Draw("le1>>h4(1000,0,10000)","cond==1&&bz==1&&dtime>5000-220*5");
h3->Sumw2();
h3->Add(h4,-1);
h3->Draw();
c.Draw();

In [28]:
tree->Draw("gea>>g9(8000,0,4000)","cond==1&&bz==1&&dtime<220*5&&le1>910&&le1<980");
tree->Draw("gea>>g10(8000,0,4000)","cond==1&&bz==1&&dtime>5000-220*5&&le1>910&&le1<980");
g9->Sumw2();
g9->Add(g10,-1);
c.Clear();
smartfit(g9,1250,1450,1350,1375,"m5","L");
g9->Draw();
c.Draw();

(TF1 *) 0x7f98991358a0
 FCN=172.254 FROM MIGRAD    STATUS=CONVERGED     109 CALLS         110 TOTAL
                     EDM=7.37181e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           2.73523e+00   4.10754e-01   2.81822e-04  -8.24123e-03
   2  p1          -1.75329e-03   3.00103e-04   2.05896e-07  -1.09466e+01
                               ERR DEF= 0.5


In [29]:
cout<<g9->Integral(2700,2750)<<'\n'<<(1+2750-2710)*(2.73523-1.75329e-03*(2700+2750.0)/4)<<endl;

538
14.2013


In [30]:
c.Clear();
h1->Draw();
smartfit(h3,600,1200,800,1100,"l4");
c.Draw();

(TF1 *) 0x7f98990a6020
 FCN=45.4509 FROM MIGRAD    STATUS=CONVERGED      34 CALLS          35 TOTAL
                     EDM=5.96644e-21    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           8.69666e+02   2.44554e+01   2.03131e-02   1.57408e-11
   2  p1           3.45646e-02   2.78832e-02   2.31603e-05   1.15047e-08


In [31]:
cout<<h3->Integral(92,98)<<'\n'<<(1+98-92)*(8.69666e+02+3.45646e-02*(92+98.0)*5)<<endl;

18910
6317.52


In [32]:
tree->Draw("gea>>g11(8000,0,4000)","cond==1&&bz==1&&dtime<220*5&&le1>2110&&le1<2200");
tree->Draw("gea>>g12(8000,0,4000)","cond==1&&bz==1&&dtime>5000-220*5&&le1>2110&&le1<2200");
g11->Sumw2();
g11->Add(g12,-1);
c.Clear();
smartfit(g11,1250,1450,1355,1375,"m6","L");
g11->Draw();
c.Draw();

(TF1 *) 0x7f989928c360
 FCN=138.641 FROM MIGRAD    STATUS=CONVERGED     173 CALLS         174 TOTAL
                     EDM=3.1524e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.9 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           3.13360e+00   4.81324e-01  -4.59465e-05  -9.82446e-04
   2  p1          -2.13246e-03   3.43912e-04   6.14578e-08  -1.16626e+00
                               ERR DEF= 0.5


In [33]:
cout<<g11->Integral(2700,2750)<<'\n'<<(1+2750-2710)*(3.13360-2.13246e-03*(2700+2750.0)/4)<<endl;

405
9.35305


In [34]:
c.Clear();
h3->Draw();
smartfit(h3,2070,2240,2080,2230,"l5");
c.Draw();

(TF1 *) 0x7f9899272480
 FCN=0.118578 FROM MIGRAD    STATUS=CONVERGED      44 CALLS          45 TOTAL
                     EDM=5.95034e-20    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.03412e+03   4.49573e+02   8.00274e-03   2.16766e-14
   2  p1          -2.11503e-01   2.10684e-01   3.75036e-06  -1.59117e-09


input_line_166:1:10: error: redefinition of 'l5'
Double_t l5(Double_t *x,Double_t *par){if(x[0]>2080.000000&&x[0]<2230.000000){TF1::RejectPoint();return 0;}retu...
         ^
input_line_112:1:10: note: previous definition is here
Double_t l5(Double_t *x,Double_t *par){if(x[0]>2090.000000&&x[0]<2230.000000){TF1::RejectPoint();return 0;}retu...
         ^


In [35]:
cout<<h3->Integral(212,220)<<'\n'<<(1+220-212)*(9.52125e+02-1.75000e-01*(220+212.0)*5)<<endl;

13644
5167.12


In [36]:
tree->Draw("gea>>g13(8000,0,4000)","cond==1&&bz==1&&dtime<220*5&&le1>3410&&le1<3520");
tree->Draw("gea>>g14(8000,0,4000)","cond==1&&bz==1&&dtime>5000-220*5&&le1>3410&&le1<3520");
g13->Sumw2();
g13->Add(g14,-1);
c.Clear();
smartfit(g13,1250,1450,1355,1375,"m7","L");
g13->Draw();
c.Draw();

(TF1 *) 0x7f98993028b0
 FCN=116.382 FROM MIGRAD    STATUS=CONVERGED     113 CALLS         114 TOTAL
                     EDM=3.62142e-11    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           7.90732e-01   2.67574e-01   1.60568e-04  -2.79758e-04
   2  p1          -4.58722e-04   1.96397e-04   1.17858e-07  -3.79358e-01
                               ERR DEF= 0.5


In [37]:
cout<<g13->Integral(2710,2750)<<'\n'<<(1+2750-2710)*( 7.90732e-01-4.58722e-04 *(2750+2710.0)/4)<<endl;

314
6.74764


In [38]:
c.Clear();
h3->Draw();
smartfit(h3,3380,3550,3390,3540,"l6");
c.Draw();

(TF1 *) 0x7f989937d930
 FCN=3.83594e-21 FROM MIGRAD    STATUS=CONVERGED      42 CALLS          43 TOTAL
                     EDM=7.67189e-21    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.02338e+03   6.97203e+02   7.85810e-03   2.12949e-15
   2  p1          -1.75000e-01   2.01168e-01   2.26735e-06  -6.08374e-10


In [39]:
cout<<h3->Integral(342,352)<<'\n'<<(1+352-342)*(1.02338e+03-1.75000e-01*(352+342.0)*5)<<endl;

11020
4577.43


In [40]:
tree->Draw("gea>>g15(8000,0,4000)","cond==1&&bz==1&&dtime<220*5&&le1>4150&&le1<4350");
tree->Draw("gea>>g16(8000,0,4000)","cond==1&&bz==1&&dtime>5000-220*5&&le1>4150&&le1<4350");
g15->Sumw2();
g15->Add(g16,-1);
c.Clear();
smartfit(g15,1250,1450,1355,1375,"m8","L");
g15->Draw();
c.Draw();

(TF1 *) 0x7f98993fde30
 FCN=191.126 FROM MIGRAD    STATUS=CONVERGED     155 CALLS         156 TOTAL
                     EDM=6.32018e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   2.5 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           5.93216e+00   7.00102e-01   1.65482e-03   8.65847e-04
   2  p1          -4.01850e-03   5.02507e-04  -1.31099e-06   1.00264e+00
                               ERR DEF= 0.5


In [41]:
cout<<g15->Integral(2710,2750)<<'\n'<<(1+2750-2710)*( 5.93216 -4.01850e-03 *(2750+2710.0)/4)<<endl;

776
18.3232


In [42]:
c.Clear();
h3->Draw();
smartfit(h3,4000,4510,4050,4450,"l7");
c.Draw();

(TF1 *) 0x7f98993f7340
 FCN=16.0839 FROM MIGRAD    STATUS=CONVERGED      37 CALLS          38 TOTAL
                     EDM=6.24881e-20    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           3.85410e+02   5.58431e+01   5.72395e-03   1.07066e-10
   2  p1          -7.46386e-02   1.29204e-02   1.32435e-06   4.76163e-07


In [43]:
cout<<h3->Integral(416,435)<<'\n'<<(1+435-416)*(3.84620e+02-7.46225e-02*(416+435.0)*5)<<endl;

26193
1342.03
